#### Import statements

In [2]:
import torch
import shutil
import os

#### Load the drive folder containing all required files

In [3]:
# mount the drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# access the drive folder containing everything we need
%cd /content/drive/My Drive/Colab environments/Risiko! DL

# check that we are in the desired folder
%ls

Mounted at /content/drive
/content/drive/My Drive/Colab environments/Risiko! DL
 3D_models/                                'Risiko! Test.ipynb'
 backgrounds/                               runs/
 coco_risiko.yaml                           synthetic_dataset/
 custom_yolo.yaml                           synthetic_images/
 datasets/                                  tanks_flags_detection.ipynb
 pre_trained_weights/                       test_example.txt
 real_images/                               yolov5/
'Risiko! Synthetic Dataset Creator.ipynb'


In [4]:
# generic path to the weights folder
weights_folder = 'runs/train'
weights_path = os.path.join(os.getcwd(), weights_folder)
print(weights_path)

# specific path to weigths obtained with 300 epochs
specific_folder = 'exp_300_epochs/weights/best.pt'
best_weights_path = os.path.join(weights_path, specific_folder)
print(best_weights_path)

/content/drive/My Drive/Colab environments/Risiko! DL/runs/train
/content/drive/My Drive/Colab environments/Risiko! DL/runs/train/exp_300_epochs/weights/best.pt


#### Clone the GitHub repository yolov5 and install requirements

In [5]:
print(os.getcwd())
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

/content/drive/My Drive/Colab environments/Risiko! DL
fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/drive/My Drive/Colab environments/Risiko! DL/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.6/599.6 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.4 MB/s eta 0:00:00


#### Model definition

In [6]:
# model
print(os.getcwd())
model = torch.hub.load(os.getcwd(), 'custom', path = best_weights_path, source ='local')

/content/drive/My Drive/Colab environments/Risiko! DL/yolov5


YOLOv5 🚀 2023-6-12 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7042489 parameters, 0 gradients, 15.9 GFLOPs
Adding AutoShape... 


####Intersection over Union (IoU)
We use this method in the section below during the evaluation of the model

In [16]:
# Method to compute the Interection over Union
def calculate_iou(box1, box2):
    # Extract coordinates of the first bounding box
    x1_1, y1_1, x2_1, y2_1, w1_1, w1_2 = box1

    # Extract coordinates of the second bounding box
    x1_2, y1_2, x2_2, y2_2, w2_1, w2_2 = box2

    # Calculate the intersection coordinates
    x_intersection = max(0, min(x2_1, x2_2) - max(x1_1, x1_2))
    y_intersection = max(0, min(y2_1, y2_2) - max(y1_1, y1_2))

    # Calculate the areas of the bounding boxes and the intersection
    area_box1 = (x2_1 - x1_1) * (y2_1 - y1_1)
    area_box2 = (x2_2 - x1_2) * (y2_2 - y1_2)
    area_intersection = x_intersection * y_intersection

    # Calculate the union area by adding the individual areas and subtracting the intersection
    area_union = area_box1 + area_box2 - area_intersection

    # Calculate the IoU by dividing the intersection area by the union area
    iou = area_intersection / (area_union + 1e-6)  # Add a small constant to avoid division by zero

    return iou

####Evaluation

In [21]:
# Set the model in evaluation mode
model.eval()

# Perform evaluation on the test set
model.conf = 0.25  # NMS confidence threshold
iou_threshold = 0.45  # IoU threshold for matching bounding boxes
epsilon = 1e-6  # Small constant for numerical stability in F1 score calculation

# Path to our dataset
path = '/content/drive/My Drive/Colab environments/Risiko! DL/datasets/test/synthetic/images'

# Get the list of files in the directory
files = os.listdir(path)

# Iterate over each file in the directory
for file_name in files:
    # Construct the full file path
    file_path = os.path.join(path, file_name)

    # Check if the current item is a file
    if os.path.isfile(file_path):
        # Evaluate the image
        results = model(file_path, size=640)

        # Extract the predicted bounding boxes and their corresponding class labels
        pred_boxes = results.xyxy[0]  # Bounding box coordinates (x1, y1, x2, y2)
        pred_labels = results.names[0]  # Predicted class labels

        # Adjust the length of pred_labels to match the number of bounding boxes
        pred_labels = pred_labels[:len(pred_boxes)]

        # Iterate over the predicted boxes
        for i in range(len(pred_boxes)):
            # Extract information for the first predicted box
            pred_box_i = pred_boxes[i]
            pred_label_i = pred_labels[i]

            # Iterate over the remaining predicted boxes
            for j in range(i + 1, len(pred_boxes)):
                # Extract information for the second predicted box
                pred_box_j = pred_boxes[j]
                pred_label_j = pred_labels[j]

                # Compute the Intersection over Union (IoU) between the two predicted boxes
                iou = calculate_iou(pred_box_i, pred_box_j)

                # Compute precision, recall, and F1 score based on the IoU
                precision = int(pred_label_i == pred_label_j)  # 1 if the predicted labels match, 0 otherwise
                recall = int(iou >= iou_threshold)  # 1 if IoU is above the threshold, 0 otherwise
                f1_score = 2 * (precision * recall) / (precision + recall + epsilon)  # Compute F1 score (add epsilon to avoid division by zero)

                print("Precision:", precision)
                print("Recall:", recall)
                print("F1 Score:", f1_score)

Precision: 0
Recall: 0
F1 Score: 0.0
Precision: 0
Recall: 0
F1 Score: 0.0
Precision: 0
Recall: 0
F1 Score: 0.0
Precision: 0
Recall: 0
F1 Score: 0.0
Precision: 0
Recall: 0
F1 Score: 0.0
Precision: 0
Recall: 0
F1 Score: 0.0
Precision: 0
Recall: 0
F1 Score: 0.0
Precision: 0
Recall: 0
F1 Score: 0.0


IndexError: ignored